# NEA Card Game

In [1]:
import sys
import random
import pandas as pd


In [2]:
user_db = pd.read_csv(r'user_list.csv')

This code reads the data from the comma seperated value file 'user_list' which contains the usernames of the players, their passwords, and their highscores.
This data is placed into a pandas dataframe for easy manipulation and handling.

In [3]:
class Player:
    
    def __init__(self, df):
        self.Score = 999
        self.Cards_Owned = []
        Username = input("Please input your username: ")
        Username.strip()
        try:
            row = df.loc[df['Username'] == Username] #creates a seperate dataframe consisting solely of the player's row in the dataframe, ensures another password is not accepted.
        except KeyError: #Alerts the user that their username is invalid
            print("Username not in database")
            return
        except:
            print("Critical System Error")
            return
        self.index = int(row.index.values)
        for x in range(0,3):#allows 3x attempts at login
            pwd = input("Please input your password: ")

            if row.at[self.index,"Password"] == pwd: #checking pwd against user row
                print("Login Successful")
                self.Username = Username
                return
            elif row.at[self.index,"Password"] != pwd:
                print("Incorrect Password")
        print("Password attempt limit exceeded.")
        sys.exit()#exits program if limit exceeded
        
    def add_score(self,inc):
        self.Score = self.Score + inc
    def init_score(self):
        self.Score = 0

The class Player has several attributes- Username (which corresponds to the Username in the 'user_list' file), index (which corresponds to the index of the player's row in the dataframe), Score (which stores the score for only a specific game), and Cards_Owned (which is a list of all the cards that the user accumulates in the game).

The login process is handled within the init function. There are also 2 extra functions, add_score and init_score which are neccesary to eliminate the potential for python to recognise self.Score as a class variable instead of an instance variable.

In [4]:
player_1 = Player(user_db)#creates instances of player

player_2 = Player(user_db)

player_1.init_score()
player_2.init_score()


Please input your username:  FXWood350
Please input your password:  NootNoot


Login Successful


Please input your username:  GTF1
Please input your password:  ProGamer


Login Successful


2 objects of the class Player are initialised.

In [5]:
deck_list = ['r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9', 'r10','y1', 'y2', 'y3', 'y4', 'y5', 'y6', 'y7', 'y8', 'y9', 'y10','b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9', 'b10']
random.shuffle(deck_list) #populates deck list and shuffles it

In [6]:
print(deck_list)

['y7', 'r2', 'b4', 'y6', 'b6', 'y10', 'y9', 'r5', 'b1', 'y3', 'b8', 'b2', 'r10', 'b5', 'y2', 'b10', 'b7', 'r6', 'y8', 'r7', 'r9', 'b3', 'r1', 'y4', 'y5', 'r4', 'r3', 'b9', 'y1', 'r8']


In [7]:
def card_select():
    global deck_list
    if len(deck_list) == 0:
        return None
    return deck_list.pop() #pulls a value from the deck, if deck is empty, returns None.

In [8]:
player_1_card = 0
player_2_card = 0

In [9]:
def turn_winner():
    global player_1
    global player_2
    letter1 = player_1_card.translate({ord(i): None for i in '1234567890'})#stores a numberless version of the card in "letter" var
    letter2 = player_2_card.translate({ord(i): None for i in '1234567890'})
    number1 = player_1_card.translate({ord(i): None for i in 'rby'})#stores a letterless version of card in "number" var
    number2 = player_2_card.translate({ord(i): None for i in 'rby'})
    
    if letter1 == letter2:
        if number1 > number2:
            player_1.Cards_Owned.append(player_1_card)
            player_1.Cards_Owned.append(player_2_card)
            player_1.add_score(2)#adds both cards to the winner's card list and increments score
            return player_1.Username
        elif number1 < number2:
            player_2.Cards_Owned.append(player_1_card)
            player_2.Cards_Owned.append(player_2_card)
            player_2.add_score(2)
            return player_2.Username
        else:
            print("Critical System Failiure.")
    
    elif (letter1 == "r" and letter2 == "b") or (letter1 == "y" and letter2 == "r") or (letter1 == "b" and letter2 == "y"): #handles all the scenarios where player 1 wins
        player_1.Cards_Owned.append(player_1_card)
        player_1.Cards_Owned.append(player_2_card)
        player_1.add_score(2)
        return player_1.Username
    else:#rest of scenarios are player 2's winnings
        player_2.Cards_Owned.append(player_1_card)
        player_2.Cards_Owned.append(player_2_card)
        player_2.add_score(2)
        return player_2.Username

In [10]:
def play_turns():
    global player_1_card
    global player_2_card
    while player_1_card != None:
        player_1_card = card_select()
        player_2_card = card_select()
        if player_1_card == None or player_2_card == None:#when the game is finished and no more cards are left to be played, the loop broken and the function returns.
            break
        print("Player 1 draws ",player_1_card," and Player 2 draws ",player_2_card)
        print(turn_winner()," wins the turn")
    return
    

In [11]:
def end_of_game():
    global player_1
    global player_2

    if player_1.Score> player_2.Score:
        print(player_1.Username," has Won!. They had a total of", str(player_1.Score), " against ", str(player_2.Score), " and hold the cards ", player_1.Cards_Owned)
        return player_1
    elif player_1.Score< player_2.Score:
        print(player_2.Username,"has Won!. They had a total of", str(player_2.Score), " against ", str(player_1.Score), " and hold the cards ", player_2.Cards_Owned)
        return player_2
    else:
        print("Draw!")
        return None

In [12]:
play_turns()
winning_player = end_of_game()#The returned object is copied into the winning_player object
if winning_player == None: #in case of a draw, no score is written to either player's highscore and the game ends
    print("It's a draw!")
    sys.exit()
user_db.loc[winning_player.index,'Highscore'] = user_db.loc[winning_player.index,'Highscore'] + winning_player.Score #The Highscore in the dataframe of the winner is overwritten with the new 
#score, with the help of the .index value of the winning player which is carried along from the original player object's value

Player 1 draws  r8  and Player 2 draws  y1
GTF1  wins the turn
Player 1 draws  b9  and Player 2 draws  r3
GTF1  wins the turn
Player 1 draws  r4  and Player 2 draws  y5
GTF1  wins the turn
Player 1 draws  y4  and Player 2 draws  r1
FXWood350  wins the turn
Player 1 draws  b3  and Player 2 draws  r9
GTF1  wins the turn
Player 1 draws  r7  and Player 2 draws  y8
GTF1  wins the turn
Player 1 draws  r6  and Player 2 draws  b7
FXWood350  wins the turn
Player 1 draws  b10  and Player 2 draws  y2
FXWood350  wins the turn
Player 1 draws  b5  and Player 2 draws  r10
GTF1  wins the turn
Player 1 draws  b2  and Player 2 draws  b8
GTF1  wins the turn
Player 1 draws  y3  and Player 2 draws  b1
GTF1  wins the turn
Player 1 draws  r5  and Player 2 draws  y9
GTF1  wins the turn
Player 1 draws  y10  and Player 2 draws  b6
GTF1  wins the turn
Player 1 draws  y6  and Player 2 draws  b4
GTF1  wins the turn
Player 1 draws  r2  and Player 2 draws  y7
GTF1  wins the turn
GTF1 has Won!. They had a total of 24

In [13]:
user_db.to_csv(r'user_list.csv', index=False) #updated database overwrites existing database
high_scores = user_db.sort_values(by='Highscore', ascending=False) #new dataframe, copy of user database, sorted by Descending Highscore value
del high_scores['Password'] #ensures other players do not find out passwords of other users
print("Highscores: \n", high_scores.head(5)) #first 5 users are printed

Highscores: 
     Username  Highscore
2     Userme       1051
0  FXWood350        666
1       GTF1        444
4       John         18
3       Dave          0
